# Algorytm genetyczny
Analiza

In [1]:
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import pandas as pd
from IPython.display import display
from enum import Enum

In [2]:
def AnalyseGEN(plot_name, files, expected_results, iterations=10,
               populationSize=10, mutationProbability = 0.05, crossesPerGeneration=50,
                drawPlot = True):
    # SYMULOWANE WYŻARZANIE
    actual_results = []
    error = []
    
    index = 0
    for f in files:
        print("solving", f, end ="...")
        
        p1 = subprocess.Popen(
            ["../Debug/PEA_Projekt.exe",
             "GEN",
             "../Debug/Data/"+f+".txt",
             str(iterations),
             str(populationSize),
             str(mutationProbability),
             str(crossesPerGeneration),
             str(0)],
            stdout=subprocess.PIPE)
        
        text_result = p1.communicate()[0].strip()
        print("result", text_result, end ="...")
        single_result = float(text_result)
        actual_results.append(single_result)
        
        index+=1
        
    errors = [(abs(expected_results[x] - actual_results[x])) / expected_results[x] * 100
                   for x in range(len(actual_results))]
    
    avgError = sum(errors) / len(errors)
    
    resultTable = pd.DataFrame(
        {'baseFile' : files,
         'actual_results': actual_results,
         'expected_results': expected_results,
         'error': errors
        })
    
    if drawPlot:
        plt.rcParams['figure.figsize'] = [20, 5]
        fig, axes = plt.subplots()

        ind = np.arange(len(expected_results))
        width = 0.4   

        plt.bar(ind, expected_results, width, label='Wartosci globalnie minimalne')
        plt.bar(ind + width, actual_results, width,
            label='Wartosci obliczone przez algorytm')

        plt.ylabel('Wagi scieżek')
        plt.title(plot_name)

        plt.xticks(ind + width / 2, files)
        plt.legend(loc='best')

        plt.show()
    
    print("avgErr", avgError)
    
    return (resultTable, avgError)

In [3]:
files = ["TSP/data17", "TSP/data21", "TSP/data24", "TSP/data26", "TSP/data29", "TSP/data42", "TSP/data58", "TSP/data120"]
expected_results = [2085, 2707, 1272, 937, 1610, 699, 25395, 6942]

iterations_of_dependency_check = 50

In [4]:
class Parameter(Enum):
    POPULATION_SIZE = 1
    MUTATION_PROBABILITY = 2
    CROSSES_PER_GENERATION = 3

def parameterImpactAnalysis(multiplier = 1,
                            parameter = Parameter.POPULATION_SIZE,
                            iterations=2,
                            file=files[3],
                            expected_result=expected_results[3]):
    """
    Wyświetla wykres ilustrujący wpływ zmiany parametru na działanie algorytmu
    """
    avg_errs = []
    values_used = []
    for i in range(iterations):
        value_used = (i+1)*multiplier
        if(parameter == Parameter.POPULATION_SIZE):
            avg_errs.append(
                AnalyseGEN("Wpływ wielkości populacji na skuteczność algorytmu",
                           [file],
                           [expected_result],
                           iterations=100,
                           populationSize=value_used,
                           drawPlot=False)[1])
        elif(parameter == Parameter.MUTATION_PROBABILITY):
            avg_errs.append(
                AnalyseGEN("Wpływ prawodopodobieństwa mutacji na skuteczność algorytmu",
                           [file],
                           [expected_result],
                           iterations=100,
                           populationSize=50,
                           mutationProbability=value_used,
                           drawPlot=False)[1])
        elif(parameter == Parameter.CROSSES_PER_GENERATION):
            avg_errs.append(
                AnalyseGEN("Wpływ ilości potomstwa na skuteczność algorytmu",
                           [file],
                           [expected_result],
                           iterations=100,
                           populationSize=50,
                           crossesPerGeneration=value_used,
                           drawPlot=False)[1])
        values_used.append(value_used)
        
    print(avg_errs)

    plot, ax = plt.subplots()
    ax.set_ylabel("Odchylenie od wartości opt. [%]")
    ax.set_xlabel("Wartość modyfikowanego parametru")
    #plot.plot(values_used, avg_errs)
    
    # trendline
    z = np.polyfit(values_used, avg_errs, 1)
    p = np.poly1d(z)
    
    plt.plot(values_used, avg_errs, values_used, p(values_used),"b--")
    plt.show()
    

## Zależność dokładności algorytmu od wielkości populacji

In [ ]:
parameterImpactAnalysis(5,
                        Parameter.POPULATION_SIZE,
                        iterations=iterations_of_dependency_check,
                        # iterations=3,
                        file=files[2],
                        expected_result=expected_results[2])

solving TSP/data24...result b'1619'...avgErr 27.27987421383648
solving TSP/data24...result b'1500'...avgErr 17.92452830188679
solving TSP/data24...result b'1664'...avgErr 30.81761006289308
solving TSP/data24...result b'1722'...avgErr 35.37735849056604
solving TSP/data24...result b'1527'...avgErr 20.047169811320757
solving TSP/data24...result b'1614'...avgErr 26.88679245283019
solving TSP/data24...result b'1504'...avgErr 18.238993710691823
solving TSP/data24...result b'1382'...avgErr 8.647798742138365
solving TSP/data24...result b'1329'...avgErr 4.481132075471698
solving TSP/data24...result b'1318'...avgErr 3.6163522012578615
solving TSP/data24...result b'1413'...avgErr 11.084905660377359
solving TSP/data24...result b'1460'...avgErr 14.779874213836477
solving TSP/data24...result b'1289'...avgErr 1.3364779874213837
solving TSP/data24...result b'1453'...avgErr 14.229559748427672
solving TSP/data24...result b'1378'...avgErr 8.333333333333332
solving TSP/data24...result b'1367'...avgErr 7.4

## Zależność dokładności algorytmu od prawdopodobieństwa mutacji

In [ ]:
parameterImpactAnalysis(0.025,
                        Parameter.MUTATION_PROBABILITY,
                        iterations=iterations_of_dependency_check,
                        file=files[2],
                        expected_result=expected_results[2])

## Zależność dokładności algorytmu od ilości potomstwa każdej z generacji 

In [ ]:
parameterImpactAnalysis(5,
                        Parameter.CROSSES_PER_GENERATION,
                        iterations=iterations_of_dependency_check,
                        file=files[2],
                        expected_result=expected_results[2])

## Porównanie wyników uzyskanych przez algorytm z wynikami optymalnymi dla przykładowych zestawów danych

In [ ]:
display(AnalyseGEN("Algorytm genetyczny", files, expected_results, iterations=100, populationSize=100)[0])